In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D,
    Reshape, Layer, BatchNormalization
)
from tensorflow.keras import Sequential

class CustomPadding(Layer):
    def __init__(self, padding_size, **kwargs):
        super(CustomPadding, self).__init__(**kwargs)
        self.padding_size = padding_size

    def call(self, inputs):
        initial_values = inputs[:, 0:1, :, :]
        final_values = inputs[:, -1:, :, :]

        initial_padding = tf.tile(initial_values, [1, self.padding_size, 1, 1])
        final_padding = tf.tile(final_values, [1, self.padding_size, 1, 1])

        return tf.concat([initial_padding, inputs, final_padding], axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + self.padding_size * 2, input_shape[2], input_shape[3])

class RemovePadding(Layer):
    def __init__(self, padding_size, **kwargs):
        super(RemovePadding, self).__init__(**kwargs)
        self.padding_size = padding_size

    def call(self, inputs):
        return inputs[:, self.padding_size:-self.padding_size, :, :]

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] - self.padding_size * 2, input_shape[2], input_shape[3])

def create_autoencoder():
    autoencoder = Sequential([
        # Pre-Processing
        Input(shape=(1000, 1)),
        Reshape((1000, 1, 1)),
        CustomPadding(padding_size=10),

        # Encoder
        Conv2D(16, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 1), padding='same'),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 1), padding='same'),

        # Bottleneck
        Conv2D(64, (3, 3), activation='relu', padding='same'),

        # Decoder
        UpSampling2D((2, 1)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        UpSampling2D((2, 1)),
        Conv2D(16, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),

        # Post-Processing
        Conv2D(1, (3, 3), activation='tanh', padding='same'),
        RemovePadding(padding_size=10),
        Reshape((1000, 1))
    ])

    return autoencoder

2023-12-14 02:13:47.710664: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 02:13:47.712844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 02:13:47.753266: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-14 02:13:47.754944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 02:13:48.708404: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [4]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(input_data, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# compile autoencoder
autoencoder = create_autoencoder()
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# train autoencoder
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=32,
                validation_data=(X_test, X_test),
                callbacks=[early_stopping])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1000, 1, 1)        0         
                                                                 
 custom_padding (CustomPadd  (None, 1020, 1, 1)        0         
 ing)                                                            
                                                                 
 conv2d (Conv2D)             (None, 1020, 1, 16)       160       
                                                                 
 batch_normalization (Batch  (None, 1020, 1, 16)       64        
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 510, 1, 16)        0         
 D)                                                              
                                                        

In [6]:
import tensorflow as tf

# save model
export_path = "./autoencoder/single_axis/accel_x"
tf.saved_model.save(autoencoder, export_path)

INFO:tensorflow:Assets written to: ./autoencoder/single_axis/accel_x/assets


INFO:tensorflow:Assets written to: ./autoencoder/single_axis/accel_x/assets
